# Week 04 Assignment weather data

Welcome to week four of this course programming 1. Analyzing time related data such as estimating seasonal effect, or year effect might be a challenge. How to filter the essential information from the noise? How to apply signal analysis with noisy data. How to make compact useful visualizations? Python has several constructs to handle date time related data. The relevant classes for making plots are Locators and Formatters. Locators determine where the ticks are, and formatters control the formatting of tick labels. The relevant class for date time data is the pandas datetime data type, which has methods like resample and several possibilities to display data (frequencies). As a study case we will work with weather data. If you have data that fits the learning goals, you can bring your own data.

Keywords: signal processing, smoothing, resample, formatters and locators, datetime object

More to read: 

- https://fennaf.gitbook.io/bfvm19prog1/
- https://machinelearningmastery.com/time-series-data-visualization-with-python/
- https://towardsdatascience.com/how-to-plot-time-series-86b5358197d6
- In the https://pandas.pydata.org/docs/reference/offset_frequency.html you can find more about frequencies and in the documentation
- https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html you can read all the methods of this datetime object.

Learning objectives

- load, inspect and clean a dataset
- reshape dataframes to group data in a certain frequency
- apply smoothing technologies
- Create useful visualisation with timeseries data
- Maintain development environment 
- Apply coding standards and FAIR principles


## Assignment

You will to organise your data into the required format and apply smoothing. In this assignment we will work with weatherdata from the KNMI. A subset of weatherdata is for you available in the file: `KNMI_20181231`. The data consist of several stations with daily weather data of several years. Your task is to make a plot similar to the plot below. 

<img src="../images/weather.png" alt="drawing" width="400"/>


Furthermore the plot needs the following enhancements

1. proper titles and ticks
2. widgets selecting a particular year or all years
3. lines need to be smoothed
3. legends needs to be added

Use your creativity. Consider colors, alpha settings, sizes etc. 

Learning outcomes

- load, inspect and clean a dataset 
- reformat dataframes
- apply smoothing technologies
- visualize timeseries data

The assignment consists of 6 parts:

- [part 1: load the data](#0)
- [part 2: clean the data](#1)
- [part 3: reformat data](#2)
- [part 4: smooth the data](#3)
- [part 5: visualize the data](#4)
- [part 6: Challenge](#5)

Part 1 and 5 are mandatory, part 6 is optional (bonus)
To pass the assingnment you need to a score of 60%. 


NB if you want to make a plot with more actual data you can download data from https://openweathermap.org/api 


---

<a name='0'></a>
## Part 1: Load the data

Either load the dataset `KNMI_20181231.csv` or `KNMI_20181231.txt.tsv`. The dataheaders contain spaces and are not very self explainable. Change this into more readable ones. Select data from a station. Station 270 is in the neighborhood of Groningen. For our plot we only need the the mean, minimum and maximum temperature. Of course you are welcome to select other data if you think it might be useful for your visualization. The data should look something like this:


In [1]:
import pandas as pd
import pathlib
import numpy as np
import re
import yaml

In [6]:
def get_filepath():
    with open('config.yaml', 'r') as stream:
        config = yaml.safe_load(stream)
        
    return config['datadir']

data_dir = get_filepath()
file_name = 'KNMI_20181231.txt.tsv'
file_path = pathlib.Path(data_dir, file_name)

with open(file_path, 'r') as f:
    data = f.readlines()

--------------

I only learned later on, that you can read a file like this, and use `comment='#'`.. So I did everything with regex as I approached it as a normal text file.

In [5]:
start_index = None

for i, line in enumerate(data):
    if 'STN,YYYYMMDD' in line:
        start_index = i + 1
        
print('The data we need starts at index {}.'.format(start_index))

The data we need starts at index 64.


In [5]:
def extract_data(data):
    """ 
    Loop through the lines in the file, add them to the dictionary, and
    convert the dictionary to a DataFrame as this is the fast method.
    https://stackoverflow.com/questions/57000903/what-is-the-fastest-and-most-efficient-way-to-append-rows-to-a-dataframe
    """
    dataset = {
        'STN': [],
        'Date': [],
        'Tmean': [],
        'Tmm': [],
        'Tmax': []
    }
    
    stn_date_pattern = '(270),([0-9]+),'
    numerical_pattern = '-{0,1}[0-9]+'

    for line in data:
        res = re.findall(stn_date_pattern, line)
        if res:
            stn, date = res[0]
            try:
                tg, tn, tx, sq, dr, rh = re.findall(numerical_pattern, line)[2:] # skip stn and date
                dataset['STN'].append(stn)
                dataset['Date'].append(date)
                dataset['Tmean'].append(tg)
                dataset['Tmm'].append(tn)
                dataset['Tmax'].append(tx)
            except ValueError:
                print('Expects six values for station {}!.'.format(stn))
                
    return pd.DataFrame(dataset)

---

<a name='1'></a>
## Part 2: Clean the data

The data ia not clean. There are empty cells in the dataframe which needs to be replaced with NaN's and the temperature is in centidegrees which needs to be transformed into degrees. The date field needs a datetime format. For visualization convience we would like to remove the leap year. Conduct the cleaning.

In [6]:
df = extract_data(data[start_index:]).astype({
    'STN': 'int',
    'Date': 'datetime64[ns]',
    'Tmean': 'float',
    'Tmm': 'float',
    'Tmax': 'float'
})
df.head()

,STN,Date,Tmean,Tmm,Tmax
0,270,2000-01-01,42.0,-4.0,79.0
1,270,2000-01-02,55.0,33.0,74.0
2,270,2000-01-03,74.0,49.0,89.0
3,270,2000-01-04,46.0,22.0,75.0
4,270,2000-01-05,41.0,14.0,56.0


In [7]:
# Check NaN
df.isna().sum()

STN      0
Date     0
Tmean    0
Tmm      0
Tmax     0
dtype: int64

In [8]:
df['Date'].dt.month

0        1
1        1
2        1
3        1
4        1
        ..
6935    12
6936    12
6937    12
6938    12
6939    12
Name: Date, Length: 6940, dtype: int64

In [9]:
# Multiply temperatures by 0.1, as the temperature was in units of 0.1.
df[['Tmean', 'Tmm', 'Tmax']] = df[['Tmean', 'Tmm', 'Tmax']].multiply(0.1)
df.head()

# used in last assignment
root = df.copy()
root = root.set_index('Date')

# remove the leap days
df = df[~((df['Date'].dt.month == 2) & (df['Date'].dt.day == 29))]
df.head()

,STN,Date,Tmean,Tmm,Tmax
0,270,2000-01-01,4.2,-0.4,7.9
1,270,2000-01-02,5.5,3.3,7.4
2,270,2000-01-03,7.4,4.9,8.9
3,270,2000-01-04,4.6,2.2,7.5
4,270,2000-01-05,4.1,1.4,5.6


In [10]:
df = df.set_index('Date')
df.head()

,STN,Tmean,Tmm,Tmax
Date,,,,
2000-01-01,270,4.2,-0.4,7.9
2000-01-02,270,5.5,3.3,7.4
2000-01-03,270,7.4,4.9,8.9
2000-01-04,270,4.6,2.2,7.5
2000-01-05,270,4.1,1.4,5.6


-----------------

This dataset contains 6935 entries, whereas the dataset should contain 24820 entries. This is caused by a difference in the file that is used.

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6935 entries, 2000-01-01 to 2018-12-31
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   STN     6935 non-null   int32  
 1   Tmean   6935 non-null   float64
 2   Tmm     6935 non-null   float64
 3   Tmax    6935 non-null   float64
dtypes: float64(3), int32(1)
memory usage: 243.8 KB


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>pd.to_datetime(df['Date'].astype(str), format='%Y%m%d')</li>
    <li>regex for empty cells = `^\s*$` </li>
    <li>remove month == 2 & day == 29</li> 
</ul>
</details>

### Expected outcome

---

<a name='2'></a>
## Part 3: Reform your data

First we will split the data in data from 2018 and data before 2018. Best is to split this in two dataframes. 
Next we need for the non 2018 data the minimum values for each day and the maximum values for each day. So we look for the minimum value out of all january-01 minimum values (regardless the year). Create a dataframe with 365 days containing the ultimate minimum and the ultimate maximum per day. 


In [12]:
df_pre = df.loc[:'2017-12-31']
df_after = df.loc['2018-01-01':]

In [13]:
def month_day(df_multipleyears):
    intersect = ['Tmm', 'Tmean', 'Tmax']
    
    df = df_multipleyears.copy()
    df = df.groupby(df.index.strftime('%m-%d')).agg({'Tmm':'min', 'Tmax':'max', 'Tmean':'mean'})    
    df['date'] = df.index
    df[['month', 'day']] = df['date'].str.split('-', 1, expand=True).astype('int')
    df = df.set_index([df['month'], df['day']]).sort_index()
    
    return df[df.columns.intersection(intersect)]

In [14]:
def test_reformed(df):
    return month_day(df)
    

test_reformed(df_pre)

Tmm  Tmax     Tmean
month day                      
1     1    -5.8  11.1  3.555556
      2    -7.5  10.2  3.538889
      3   -12.6  10.7  3.144444
      4    -6.7   9.8  3.244444
      5    -6.2   9.4  2.766667
...         ...   ...       ...
12    27   -6.0  11.7  3.633333
      28   -7.4  10.9  3.622222
      29   -7.3  11.4  2.805556
      30  -10.2  11.1  3.016667
      31  -10.6  11.8  3.511111

[365 rows x 3 columns]

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use the dt.month and dt.day to groupby</li>
</ul>
</details>

### Expected outcome
Note, the layout or names my differ, but the length should be 365 and the minimum values should be the same

---

<a name='3'></a>
## Part 4: Smooth the data

Make a function that takes an array or a dataframe column and returns an array of smoothed data. Explain in words why you choose a certain smoothing algoritm. Ask the signal analysis teacher if you want some advice.


----------

**NOTE**: What is the best practice here, returning a view or a copy? I myself thought a copy would be better, to make sure the original one it not being modified. However, I get a warning.

Why EWM and MA? Because they are often used in time series analysis of stocks. But, are they applicable to weather data ? Due to the seasonal effect of weather data it is better to use other types of moving average. Another factor that plays a roll in weather data, is the long term trend.

A great resource: https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2004GL019569

Unfortunately, my knowledge is too little to implement the algorithms Michael Mann proposes. He uses, for instance, the butterworth smoothing filter and some specific constraints. For practical reasons I'll use the exponential moving average, the simple moving average and the fourier transform.

In [15]:
def smooth_data(df, col, periods):
    """returns EWM, MA and Fourier Transform"""
    df = df.copy()
    start_vals = df[col].iloc[:periods]
    
    df['ewm'] = df[col].ewm(com=0.5, min_periods=periods).mean()
    df['ewm'].iloc[:periods] = start_vals
    df['ewm'] = df['ewm'].round(2)
    
    df['ma'] = df[col].rolling(periods).mean()
    df['ma'].iloc[:periods] = start_vals
    df['ma'] = df['ma'].round(2)
    
    n = df.index.size
    y = df[col]
    
    rft = np.fft.rfft(y)
    rft[5:] = 0
    y_smooth = np.fft.irfft(rft, n)
    
    df['ft'] = y_smooth
    
    return df

In [16]:
values_df = test_reformed(df_pre)
values_df = smooth_data(values_df, 'Tmean', 7)

---

<a name='4'></a>
## Part 5: Visualize the data

Plot the mean temperature of the year 2018. Create a shaded band with the ultimate minimum values and the ultimate maximum values from the multi-year dataset. Add labels, titles and legends. Use proper ranges. Be creative to make the plot attractive. 



<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use from bokeh.models import Band</li>
    <li>use ColumnDataSource to parse data arrays</li>
    <li>look for xaxis tick formatters</li>
</ul>
</details>

In [17]:
from bokeh.models import ColumnDataSource, Band, Legend, HoverTool, Slider
from bokeh.models.tools import CustomJSHover
from bokeh.plotting import Figure, figure, show, output_notebook,gridplot
from bokeh.themes import built_in_themes
from bokeh.io import curdoc
from bokeh.layouts import column

curdoc().theme = 'dark_minimal'

output_notebook()

Loading BokehJS ...

In [18]:
df_after = smooth_data(df_after, 'Tmean', 7)
df_after.tail()

C:\Users\ddlat\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,STN,Tmean,Tmm,Tmax,ewm,ma,ft
Date,,,,,,,
2018-12-27,270,5.7,5.3,6.2,6.04,6.50,5.279558
2018-12-28,270,7.1,5.8,8.1,6.75,6.40,5.297463
2018-12-29,270,8.5,6.9,10.2,7.92,6.56,5.313009
2018-12-30,270,8.0,6.8,9.0,7.97,6.77,5.325799
2018-12-31,270,8.7,8.2,9.7,8.46,7.34,5.335445


In [19]:
source = ColumnDataSource(df_after)

lowest = df_after['Tmean'].min()
highest = df_after['Tmean'].max()

p = figure(title='Average temperature in 2018', x_axis_type='datetime', width=1000)
p.line(x='Date', y='Tmean', source=source, legend_label='Average', color='blue')
p.line(x='Date', y='ma', source=source, legend_label='Moving Average', color='red')
p.line(x='Date', y='ewm', source=source, legend_label='Exponential MA', color='green')
p.line(x='Date', y='ft', source=source, legend_label='Fourier Transform', color='yellow')


band = Band(base='Date', lower='Tmm', upper='Tmax', source=source, level='underlay',
           fill_alpha=0.1, fill_color='lightgrey', line_width=1, line_color='salmon')
p.add_layout(band)

p.title.text = "Temperature 2018 Smoothed"
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Degrees in Celcius'

p.legend.click_policy="hide"
p.legend.location = 'top_left'

p.add_tools(
    HoverTool(
        show_arrow=False,
        line_policy='next',
        tooltips=[
            ('Avg', '@Tmean'),
            ('Max', '@Tmax'),
            ('Min', '@Tmm')
        ],
    )
)

show(p)

---

<a name='5'></a>
## Part 6: Challenge

Make a widget in which you can select the year range for the multiyear set. Or maybe a widget were you choose a different station. Add this to your layout to make the plot interactive. Add another widget to select or deselect the smoother. Inspiration: https://demo.bokeh.org/weather

In [20]:
import pandas as pd
import panel as pn
from bokeh.models import ColumnDataSource, Band, HoverTool
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.plotting import figure, output_notebook
from bokeh.io import curdoc

pn.extension()

In [21]:
df = smooth_data(root, 'Tmean', 7)
df.head()

C:\Users\ddlat\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,STN,Tmean,Tmm,Tmax,ewm,ma,ft
Date,,,,,,,
2000-01-01,270,4.2,-0.4,7.9,4.2,4.2,10.240148
2000-01-02,270,5.5,3.3,7.4,5.5,5.5,10.239972
2000-01-03,270,7.4,4.9,8.9,7.4,7.4,10.239797
2000-01-04,270,4.6,2.2,7.5,4.6,4.6,10.239623
2000-01-05,270,4.1,1.4,5.6,4.1,4.1,10.239450


In [22]:
start_date = df.index[0]
inter_date = df.index[364] # 1 year is shown by default
end_date = df.index[-1]

date_range_slider = pn.widgets.DateRangeSlider(
    name='Date Range Slider',
    start=start_date,
    end=end_date,
    value=(start_date, inter_date)
)

select = pn.widgets.Select(name='Select', options=['Average', 'MA', 'EWMA'])

In [23]:
plot_data = ColumnDataSource(df)

In [24]:
def select_smoothing(method, window):
    # Update the local truth with a selection of the global truth

    start, end = window
    temp = df[start:end]
    if method == 'MA':
        plot_data.data['Tmean'] = temp['ma']
    elif method == 'EWMA':
        plot_data.data['Tmean'] = temp['ewm']
    else:
        plot_data.data['Tmean'] = temp['Tmean']
        
def select_date(window):
    # Update the local truth with a selection of the global truth
    start, end = window
    plot_data.data = df[start:end]

In [25]:
# The leap years are removed.

p = figure(title='Average Temperature', x_axis_type='datetime')

p.line(x='Date', y='Tmean', source=plot_data, legend_label='Average temperature')
band = Band(base='Date', lower='Tmm', upper='Tmax', source=plot_data,
            level='underlay', fill_alpha=0.1, fill_color='grey',
            line_width=1, line_color='salmon')
p.add_layout(band)

p.title.text = "Temperature over the years"
p.xaxis.axis_label = 'Date'

p.xaxis.formatter = DatetimeTickFormatter(months='%b %Y')
p.yaxis.axis_label = 'Temperature in °C'

p.legend.location = 'top_left'

p.add_tools(
    HoverTool(
        show_arrow=False,
        line_policy='next',
        tooltips=[
            ('Avg', '@Tmean'),
            ('Max', '@Tmax'),
            ('Min', '@Tmm')
        ],
    )
)

pane = pn.Column(date_range_slider, select, pn.pane.Bokeh(p))
pn.interact(select_date, window=date_range_slider)
pn.interact(select_smoothing, method=select, window=date_range_slider)

pane

Column
    [0] DateRangeSlider(end=Timestamp('2018-12-31 0..., name='Date Range Slider', start=Timestamp('2000-01-01 0..., value=(Timestamp('2000-01-01 00:..., value_end=Timestamp('2000-12-30 0..., value_start=Timestamp('2000-01-01 0...)
    [1] Select(options=['Average', 'MA', 'EWMA'], value='Average')
    [2] Bokeh(Figure)